# Lesson 4

Part 1
In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on: - Null hypothesis - Alternate hypothesis - Level of significance - Test statistic - P-value - F table

Context
Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data anova_lab_data.xlsx file in the files_for_lab folder

State the null hypothesis
State the alternate hypothesis
What is the significance level
What are the degrees of freedom of model, error terms, and total DoF

1. State the null hypothesis: 
    H0 = the average etching rate is the same for different level of power

2. State the alternate hypothesis: 
    Ha = The average etching rates are different when the power changes

3. What is the significance level: 
    95% 
    α = 0.05

4. What are the degrees of freedom of model, error terms, and total DoF: 
    
    DoF num = 3-1 = 2 
    Dof denominator = total number of values minus 1 15-3 = 12 
    critical value = 3.8853

## Loading libraries

In [1]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
import numpy as np
import researchpy as rp
import scipy.stats as stats

## Example 1

In [2]:
data = pd.read_excel('anova_lab_data.xlsx', sheet_name='data_collected')
data

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Power         15 non-null     object 
 1   Etching Rate  15 non-null     float64
dtypes: float64(1), object(1)
memory usage: 368.0+ bytes


In [4]:
data.columns #'Power' has a space after it, we will need to rename the column

Index(['Power ', 'Etching Rate'], dtype='object')

In [5]:
data.columns = ['Power','Etching_Rate']

In [6]:
rp.summary_cont(data['Etching_Rate'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,Etching_Rate,15.0,6.7827,1.2286,0.3172,6.1023,7.4631


Manually:

We need to compute:

$$F=\frac{\hat{S}_{t}^{2}}{\hat{S}_{E}^{2}}=\frac{\frac{\sum_{k}n_{k}(\bar{y}_{k}-\bar{y})^{2}}{(k-1)}}{\frac{\sum_{k,i}(y_{k,i}-\bar{y}_{k})^{2}}{(N-k)}}$$

We start computing the global mean and the means of each group.

In [8]:
import numpy as np

# Global mean:
gm = np.mean(data['Etching_Rate'])
print("Global mean",gm)

# Mean of every group:
group_averages = [(group,np.mean(data[data['Power'] == group]['Etching_Rate'])) for group in list(data['Power'].unique())] 
print(group_averages)

Global mean 6.782666666666667
[('160 W', 5.792), ('180 W', 6.2379999999999995), ('200 W', 8.318)]


Now we can compute the numerator of F:

In [9]:
numerator = 0
for group in group_averages:
    numerator += len(data[data['Power'] == group[0]]['Etching_Rate'].values) * ( group[1] - gm )**2
numerator = numerator / ( len(data['Power'].unique()) - 1 ) 

print("\hat{S}_{t}^{2} is: ", numerator)

\hat{S}_{t}^{2} is:  9.088326666666667


Now the denominator:

In [11]:
denominator = 0
for group in group_averages: 
    for value in data[data['Power'] == group[0]]['Etching_Rate'].values:
        denominator += (value - group[1])**2
denominator = denominator / ( len(data) - len(data['Power'].unique()) )

print("\hat{S}_{e}^{2} is: ",denominator)

\hat{S}_{e}^{2} is:  0.24643666666666644


Now we can compute the ratio:

In [12]:
F = numerator / denominator

print("The F statistic is: ",F)

The F statistic is:  36.87895470100503


Now let's get the critical value for F. 

Assuming a confidence interval of 95%, our significance level $\alpha$ = 0.05. The degrees of freedom in the numerator is: (number of groups - 1) = 4 - 1 = 3. The number of degrees of freedom for the denominator is: ( number of records - number of groups ) = 20 - 4 = 16. 

If we go to the [F-Table](http://www.socr.ucla.edu/Applets.dir/F_Table.html), we get that the critical value for F is: 3.2389


In [7]:
stats.f_oneway(data['Etching_Rate'][data['Power'] == '160 W'],
               data['Etching_Rate'][data['Power'] == '180 W'],
               data['Etching_Rate'][data['Power'] == '200 W'])


F_onewayResult(statistic=36.87895470100505, pvalue=7.506584272358903e-06)